In [5]:
import tensorflow as tf
import tensorflow.keras as K
import numpy as np
import tensorflow_datasets as tfds
import cv2
from IPython.display import clear_output
import time

In [6]:
class convbnrelu(K.layers.Layer):
    def __init__(self, filters, kernels=(3, 3), strides=1, padding="SAME", data_format="NHWC", dilations=1, use_bias=False):
        super(convbnrelu, self).__init__()
        self.filters = filters
        self.kernels = kernels
        self.strides = strides
        self.padding = padding
        self.data_format = data_format
        self.dilations = dilations
        self.use_bias = use_bias

    def build(self, inputs_shape):
        if self.data_format == 'NHWC':
            channel_index = -1
            data_format_keras = 'channels_last'
        elif self.data_format == 'NCHW':
            channel_index = 1
            data_format_keras = 'channels_first'
        self.bn = K.layers.BatchNormalization(axis=channel_index)
        self.relu = K.layers.ReLU()
        self.conv = K.layers.Conv2D(filters=self.filters, 
                                    kernel_size=self.kernels, 
                                    strides=self.strides, 
                                    padding=self.padding.lower(), 
                                    data_format=data_format_keras, 
                                    use_bias=False)
    def call(self, inputs):
        tensor = self.conv(inputs)
        tensor = self.bn(tensor)
        tensor = self.relu(tensor)
        return tensor

In [7]:
class dwconvbnrelu(K.layers.Layer):
    def __init__(self, depth_multiplier=1, kernels=(3, 3), strides=1, padding="SAME", data_format="NHWC", dilations=1, use_bias=False):
        super(dwconvbnrelu, self).__init__()
        self.depth_multiplier = depth_multiplier
        self.kernels = kernels
        self.strides = strides
        self.padding = padding
        self.data_format = data_format
        self.dilations = dilations
        self.use_bias = use_bias

    def build(self, inputs_shape):
        if self.data_format == 'NHWC':
            channel_index = -1
            data_format_keras = 'channels_last'
        elif self.data_format == 'NCHW':
            channel_index = 1
            data_format_keras = 'channels_first'
        self.bn = K.layers.BatchNormalization(axis=channel_index)
        self.relu = K.layers.ReLU()
        self.conv = K.layers.DepthwiseConv2D(kernel_size=self.kernels, 
                                             strides=self.strides, 
                                             padding=self.padding.lower(), 
                                             data_format=data_format_keras, 
                                             depth_multiplier=self.depth_multiplier,
                                             use_bias=False)
    def call(self, inputs):
        tensor = self.conv(inputs)
        tensor = self.bn(tensor)
        tensor = self.relu(tensor)
        return tensor

In [8]:
class combconv(K.layers.Layer):
    def __init__(self, filters, kernels=(3, 3), strides=1, padding="SAME", data_format="NHWC", dilations=1, use_bias=False):
        super(combconv, self).__init__()
        self.filters = filters
        self.kernels = kernels
        self.strides = strides
        self.padding = padding
        self.data_format = data_format
        self.dilations = dilations
        self.use_bias = use_bias

    def build(self, inputs_shape):
        if self.data_format == 'NHWC':
            channel_index = -1
            data_format_keras = 'channels_last'
        elif self.data_format == 'NCHW':
            channel_index = 1
            data_format_keras = 'channels_first'
        self.dw = dwconvbnrelu(kernels=self.kernels, strides=self.strides, padding=self.padding, data_format=self.data_format)
        self.pw = convbnrelu(self.filters, kernels=(1, 1), padding=self.padding, data_format=self.data_format)

    def call(self, inputs):
        tensor = self.pw(inputs)
        tensor = self.dw(tensor)
        return tensor

In [9]:
class hardblock(K.Model):
    def get_link(self, layer, base_ch, growth_rate, grmul):
        if layer == 0:
            return base_ch, []
        out_channels = growth_rate
        link = []
        for i in range(10):
            dv = 2 ** i
            if layer % dv == 0:
                k = layer - dv
                link.append(k)
                if i > 0:
                    out_channels *= grmul
        out_channels = int(int(out_channels + 1) / 2) * 2
        return out_channels, link
    
    def __init__(self, base_ch, growth_rate, grmul, n_layers, dwconv=True, keepBase=False):
        super(hardblock, self).__init__()
        self.links = []
        layers_ = []
        self.keepBase = keepBase
        self.out_channels = 0
        for i in range(1, n_layers + 1):
            outch, link = self.get_link(i, base_ch, growth_rate, grmul)
            self.links.append(link)
            if dwconv:
                layers_.append(combconv(outch))
            else:
                layers_.append(convbnrelu(outch))

            if (i % 2 == 0) or (i == n_layers - 1):
                self.out_channels += outch
        self.layers_list = layers_
        self.concatenate = K.layers.Concatenate(axis=-1)
    
    def get_out_ch(self):
        return self.out_channels
        
    def call(self, x):
        layers_ = [x]
        
        for layer in range(len(self.layers_list)):
            link = self.links[layer]
            tin = []
            for i in link:
                tin.append(layers_[i])
            if len(tin) > 1:
                x = self.concatenate(tin)
            else:
                x = tin[0]
            out = self.layers_list[layer](x)
            layers_.append(out)
            
        t = len(layers_)
        out_ = []
        for i in range(t):
          if (i == 0 and self.keepBase) or \
             (i == t-1) or (i%2 == 1):
              out_.append(layers_[i])
        out = self.concatenate(out_)
        return out

In [10]:
# my_layer = combconv(512, data_format='NHWC')
# x = K.layers.Input(shape=(3, 512, 512))|

In [11]:
# my_hblock = hardblock(3, 1.7, 2, 12, keepBase=False, dwconv=False)

In [12]:
# model = K.Model(inputs=x, outputs=my_hblock(x))

In [13]:
# model.summary()

In [14]:
class HarDNet(K.Model):
    def __init__(self, depth_wise=False, arch=85, pretrained=True, num_classes=10, weight_path=''):
        super().__init__()
        first_ch  = [32, 64]
        second_kernel = 3
        max_pool = True
        grmul = 1.7
        drop_rate = 0.1
        
        #HarDNet68
        ch_list = [  128, 256, 320, 640, 1024]
        gr       = [  14, 16, 20, 40,160]
        n_layers = [   8, 16, 16, 16,  4]
        downSamp = [   1,  0,  1,  1,  0]
        
        if arch==85:
            #HarDNet85
            first_ch  = [48, 96]
            ch_list = [  192, 256, 320, 480, 720, 1280]
            gr       = [  24,  24,  28,  36,  48, 256]
            n_layers = [   8,  16,  16,  16,  16,   4]
            downSamp = [   1,   0,   1,   0,   1,   0]
            drop_rate = 0.2
        elif arch==39:
            #HarDNet39
            first_ch  = [24, 48]
            ch_list = [  96, 320, 640, 1024]
            grmul = 1.6
            gr       = [  16,  20, 64, 160]
            n_layers = [   4,  16,  8,   4]
            downSamp = [   1,   1,  1,   0]
          
        if depth_wise:
            second_kernel = 1
            max_pool = False
            drop_rate = 0.05
        
        blks = len(n_layers)
        self.base = []

        # First Layer: Standard Conv3x3, Stride=2
        self.base.append (
             convbnrelu(filters=first_ch[0], kernels=3,
                       strides=2) )
  
        # Second Layer
        self.base.append ( convbnrelu(first_ch[1],  kernels=second_kernel) )
        
        # Maxpooling or DWConv3x3 downsampling
        if max_pool:
            self.base.append(K.layers.MaxPool2D(pool_size=(3, 3), strides=2, padding='same', data_format='channels_last'))
        else:
            self.base.append ( dwconvbnrelu(strides=2) )

        # Build all HarDNet blocks
        ch = first_ch[1]
        for i in range(blks):
            blk = hardblock(ch, gr[i], grmul, n_layers[i], dwconv=depth_wise)
            ch = blk.get_out_ch()
            self.base.append ( blk )
            
            if i == blks-1 and arch == 85:
                self.base.append(K.layers.Dropout(0.1))
            
            self.base.append ( convbnrelu(ch_list[i], kernels=1) )
            ch = ch_list[i]
            if downSamp[i] == 1:
                if max_pool:
                    self.base.append(K.layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='same', data_format='channels_last'))
                else:
                    self.base.append (dwconvbnrelu(strides=2))
        self.base.append(K.layers.GlobalAveragePooling2D('channels_last'))
        self.base.append(K.layers.Flatten())
        self.base.append(K.layers.Dropout(drop_rate))
        self.base.append(K.layers.Dense(1000))
        self.base.append(K.layers.Dense(num_classes))
          
    def call(self, x):
        y = x
        for layer in self.base:
            y = layer(y)
        return y

In [15]:
# model = K.applications.resnet_v2.ResNet50V2(weights=None, classes=10, input_shape=(28, 28, 1))

In [16]:
# model.summary()
model = HarDNet(arch=39)
# model.build((None, 28, 28, 1))

In [17]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
def get_loss(logits, labels):
    probs = K.activations.softmax(logits, axis=-1)
    loss = K.losses.CategoricalCrossentropy()(probs, labels)
    return tf.reduce_mean(loss)

def train_step(input_imgs, labels, model, optimizer):
    with tf.GradientTape() as tape:
        logits = model(input_imgs)
        loss = tf.reduce_mean(K.losses.categorical_crossentropy(labels, logits, from_logits=True))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

In [18]:
fmnist = tfds.load("fashion_mnist", split="train", with_info=False)

In [19]:
fmnist_test = tfds.load("fashion_mnist", split="test", with_info=False).shuffle(1000).batch(24, drop_remainder=True)

In [20]:
def calc_metrics(model, test_data):
    print("Calculating Validation Accuracy...")
    accuracy = K.metrics.Accuracy()
    for test_batch in test_data:
        pred = tf.argmax(model(test_batch['image']/1), axis=-1)
        lab = test_batch['label']
        accuracy.update_state(lab, pred)
    print("Accuracy: ", accuracy.result().numpy())

In [21]:
# calc_metrics(model, fmnist_test)

In [22]:
# fmnist_test = fmnist_test.unbatch()

In [23]:
# for n, mini_batch in enumerate(fmnist_test):
#     val_loss += get_loss(model(mini_batch['image']/1), tf.one_hot(mini_batch['label'], 10))
# print("Validation Loss: ", val_loss/(n+1))

In [24]:
fmnist_new = fmnist.shuffle(1024).batch(24)

In [25]:
for n, mini_batch in enumerate(zip(fmnist_new, fmnist_test.repeat())):
    pass
print(n)

2499


In [26]:
logdir = "/home/badar/Code/Badr_AI_Repo/logs"
epochs = 5
batch_size = 24
n_classes = 10
total_steps = 0

In [27]:
writer = tf.summary.create_file_writer(logdir)
writer.set_as_default()

In [28]:
for epoch in range(epochs):
    for step, (mini_batch, val_mini_batch) in enumerate(zip(fmnist_new, fmnist_test.repeat().shuffle(1024, reshuffle_each_iteration=True))):
        loss = train_step(tf.cast(mini_batch['image'], tf.float32), tf.one_hot(mini_batch['label'], 10), model, optimizer)
        val_loss = get_loss(model(val_mini_batch['image']/1), tf.one_hot(mini_batch['label'], 10))
        print("Epoch {}: {}/{}, Loss: {} Val Loss: {}".format(epoch, step*batch_size, 60000, loss.numpy(), val_loss.numpy()))
        tf.summary.scalar("loss", loss, step=total_steps+step)
        tf.summary.scalar("val_loss", val_loss, step=total_steps+step)
        clear_output(wait=True)
    total_steps += (step + 1)
#     val_loss = 0
#     print("Calculating Val Loss...")
#     for n, mini_batch in enumerate(fmnist_test):
#         val_loss += get_loss(model(mini_batch['image']/1), tf.one_hot(mini_batch['label'], 10))
#     print("Validation Loss: ", (val_loss/(n+1)).numpy())
#     calc_metrics(model, fmnist_test)
#     fmnist_new = fmnist_new.shuffle(1000)
#     time.sleep(5)

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above. [Op:Conv2D]

In [28]:
for x in fmnist_new.take(1):
    print(tf.argmax(model(x['image']/1), axis=-1))
    print(x['label'])

tf.Tensor([8 0 8 5 8 1 9 4 1 9 7 1 3 9 4 7 1 9 5 0 2 7 5 7], shape=(24,), dtype=int64)
tf.Tensor([8 0 1 5 8 1 9 3 1 9 7 1 3 9 4 7 1 9 5 0 2 7 5 9], shape=(24,), dtype=int64)
